In [ ]:
!pip install -q keras-nlp --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import keras_nlp
import keras
import tensorflow as tf
import os
import gc

keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
DATA_DIR = '/kaggle/input/contradictory-my-dear-watson/'
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [ ]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [ ]:
def split_labels(x, y):
    return (x[0], x[1]), y

- Use this block if the preprocessor used for the model is changed from bert base multi

In [ ]:
batch_size = 28
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

Size of the train dataset: 12120


In [18]:
electra_discriminator_token = keras_nlp.models.Tokenizer.from_preset("electra_base_discriminator_uncased_en")
bert_preprocessor = keras_nlp.models.BertTextClassifierPreprocessor(electra_discriminator_token, sequence_length=240)
train_set = (
    train_preprocessed.map(bert_preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

electra_discriminator_back = keras_nlp.models.Backbone.from_preset("electra_base_discriminator_uncased_en")
classifier = keras_nlp.models.BertClassifier(electra_discriminator_back, 3, preprocessor=None)


classifier_history = classifier.fit(train_set, epochs=5)
training_losses = classifier_history.history["loss"]



Epoch 1/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 100s 63ms/step - loss: 1.0024 - sparse_categorical_accuracy: 0.4421
Epoch 2/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 27s 63ms/step - loss: 0.7413 - sparse_categorical_accuracy: 0.6347
Epoch 3/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 27s 63ms/step - loss: 0.6001 - sparse_categorical_accuracy: 0.6955
Epoch 4/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 27s 63ms/step - loss: 0.5172 - sparse_categorical_accuracy: 0.7319
Epoch 5/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 27s 63ms/step - loss: 0.4858 - sparse_categorical_accuracy: 0.7462


### Precitions on test data for submission

In [19]:
df_test = pd.read_csv("test.csv")
print(df_test.head())
df_test["label"] = [0] * len(df_test)

           id                                            premise  \
0  c6d58c3f69  بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...   
1  cefcc82292                             هذا هو ما تم نصحنا به.   
2  e98005252c  et cela est en grande partie dû au fait que le...   
3  58518c10ba                   与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp   
4  c32b0d16df                              Она все еще была там.   

                                          hypothesis lang_abv language  
0  کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...       ur     Urdu  
1  عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...       ar   Arabic  
2                             Les mères se droguent.       fr   French  
3                            IMA与其他组织合作，因为它们都依靠共享资金。       zh  Chinese  
4     Мы думали, что она ушла, однако, она осталась.       ru  Russian  


In [20]:
def split_labels(x, y):
    return (x[0], x[1]), y

testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=265)

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(1, drop_remainder=False).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))




5195


In [21]:
predictions = classifier.predict(bert_test_set)

5195/5195 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step


In [22]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

[2 2 2 ... 2 2 2]


In [23]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,2
2,e98005252c,2
3,58518c10ba,1
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,2
5191,f357a04e86,2
5192,1f0ea92118,2
5193,0407b48afb,2
